In [1]:
import numpy as np

In [2]:
# Оценка сходства двух товаров по косинусной мере
def item_similarity(data, item_a, item_b):
    row_a = data[item_a, :]
    row_b = data[item_b, :]

    mask = (row_a != 0) & (row_b != 0)
    if mask.sum() == 0:
        return 0.0

    v1 = row_a[mask]
    v2 = row_b[mask]

    numerator = np.dot(v1, v2)
    denominator = np.linalg.norm(v1) * np.linalg.norm(v2)
    return numerator / denominator if denominator != 0 else 0.0


# Формируем матрицу сходства товаров
def compute_item_sim_matrix(data):
    n = data.shape[0]
    sim_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(i + 1, n):
            value = item_similarity(data, i, j)
            sim_matrix[i, j] = value
            sim_matrix[j, i] = value
    return sim_matrix


# Средняя оценка пользователя
def user_mean(data, user):
    scores = data[:, user]
    scores = scores[scores != 0]
    return scores.mean() if scores.size else 0.0


In [3]:
# Предсказание оценки item-based
def estimate_score(data, similarities, user, target_item, k=2):
    rated_items = [i for i in range(data.shape[0]) if i != target_item and data[i, user] != 0]
    if not rated_items:
        return None, []

    sims = np.array([similarities[target_item, i] for i in rated_items])
    k = min(k, len(rated_items))
    best_idx = np.argsort(-sims)[:k]
    neighbors = [rated_items[i] for i in best_idx]

    numerator = 0.0
    denominator = 0.0

    for item in neighbors:
        w = similarities[target_item, item]
        numerator += w * data[item, user]
        denominator += abs(w)

    if denominator == 0:
        return None, neighbors

    return numerator / denominator, neighbors

In [4]:
# Товарная популярность
def most_popular_item(data):
    means = np.array([
        row[row != 0].mean() if np.any(row != 0) else 0
        for row in data
    ])
    return means.argmax(), means

In [5]:
def recommend_item(data, similarities, user, item, k=2, threshold=4.0, user_labels=None, item_labels=None):
    uname = user_labels[user] if user_labels else f"User{user+1}"

    if np.all(data[:, user] == 0):
        idx, means = most_popular_item(data)
        iname = item_labels[idx] if item_labels else f"Item{idx+1}"
        print(f"\n{uname} — новый пользователь → рекомендуем популярное: {iname} ({means[idx]:.2f})")
        return

    iname = item_labels[item] if item_labels else f"Item{item+1}"
    score, neigh = estimate_score(data, similarities, user, item, k)

    print(f"\nОценка для {uname} по {iname}:")
    if score is None:
        print("Недостаточно данных для прогноза")
        return

    print("Похожие товары:", [item_labels[n] for n in neigh] if item_labels else neigh)
    print(f"Прогнозируемая оценка: {score:.3f}")

    avg = user_mean(data, user)
    print(f"Средняя оценка пользователя: {avg:.3f}")

    if score >= threshold and score >= avg:
        print(f"РЕКОМЕНДУЕТСЯ: {score:.2f} ≥ {threshold} и ≥ {avg:.2f}")
    else:
        print(f"НЕ рекомендуется: {score:.2f} < {threshold} или < {avg:.2f}")

In [6]:
data = np.array([
    [4, 0, 3, 5, 4],
    [0, 4, 5, 0, 3],
    [5, 5, 0, 4, 0],
    [3, 4, 2, 0, 5],
    [0, 3, 0, 4, 4],
])

users = ["U1", "U2", "U3", "U4", "U5"]
items = ["P1", "P2", "P3", "P4", "P5"]

print("Рейтинги:")
print(data, "\n")

best_item, means = most_popular_item(data)
print("Средние оценки товаров:", np.round(means, 3))
print(f"Самый популярный товар: {items[best_item]}\n")

sim_items = compute_item_sim_matrix(data)
print("Матрица сходства товаров:")
print(np.round(sim_items, 3), "\n")

# Примеры рекомендации
recommend_item(data, sim_items, user=1, item=3, user_labels=users, item_labels=items)
recommend_item(data, sim_items, user=4, item=2, user_labels=users, item_labels=items)

Рейтинги:
[[4 0 3 5 4]
 [0 4 5 0 3]
 [5 5 0 4 0]
 [3 4 2 0 5]
 [0 3 0 4 4]] 

Средние оценки товаров: [4.    4.    4.667 3.5   3.667]
Самый популярный товар: P3

Матрица сходства товаров:
[[0.    0.926 0.976 0.963 0.994]
 [0.926 0.    1.    0.864 0.96 ]
 [0.976 1.    0.    0.99  0.968]
 [0.963 0.864 0.99  0.    1.   ]
 [0.994 0.96  0.968 1.    0.   ]] 


Оценка для U2 по P4:
Похожие товары: ['P5', 'P3']
Прогнозируемая оценка: 3.995
Средняя оценка пользователя: 4.000
НЕ рекомендуется: 4.00 < 4.0 или < 4.00

Оценка для U5 по P3:
Похожие товары: ['P2', 'P4']
Прогнозируемая оценка: 3.995
Средняя оценка пользователя: 4.000
НЕ рекомендуется: 3.99 < 4.0 или < 4.00
